# Artificial Neural Network: NBA Player Dataset Team Optimazitation\n",

## Abstract\n

The objective is to use a deep artificial neural network (ANN) to determine an optimal team composition from a pool of basketball players. Given player characteristics, we want to identify the best five players that result in a balanced team.

### Data Preparation:

* Load the NBA Players Dataset.
* Filter to get a pool of 100 players from a random 5-year window.
* Normalize/Standardize player characteristics.

### ANN Model Building:

* Design a Multi-layer Perceptron (MLP) based on the architecture of the CST-435 An Artificial Neural Network Model Image (see below)
* Define layers: Input layer, Hidden layers, and Output layer.
* Determine the appropriate activation function, optimizer, and loss function for the MLP.

![ANNModel](ANNModel.png)

### Training the ANN:

* Forward propagation: Use player characteristics to propagate input data through the network and generate an output.
* Calculate the error using a predefined cost function.
* Backpropagate the error to update model weights.
* Repeat the above steps for several epochs.

### Evaluation and Team Selection:

* Use forward propagation on the trained ANN to predict player effectiveness or class labels.
* Apply a threshold function to these predictions.
* Select the top five players that meet the optimal team criteria.

In [9]:
import pandas as pd
import random


In [10]:

# Specify the file path
file_path = "all_seasons.csv"

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(file_path)

# Display the head (first few rows) of the DataFrame
print("Head of the DataFrame:")
print(df.head())

# Display the tail (last few rows) of the DataFrame
print("\nTail of the DataFrame:")
print(df.tail())


Head of the DataFrame:
   Unnamed: 0        player_name team_abbreviation   age  player_height  \
0           0      Dennis Rodman               CHI  36.0         198.12   
1           1  Dwayne Schintzius               LAC  28.0         215.90   
2           2       Earl Cureton               TOR  39.0         205.74   
3           3        Ed O'Bannon               DAL  24.0         203.20   
4           4        Ed Pinckney               MIA  34.0         205.74   

   player_weight                      college country draft_year draft_round  \
0      99.790240  Southeastern Oklahoma State     USA       1986           2   
1     117.933920                      Florida     USA       1990           1   
2      95.254320                Detroit Mercy     USA       1979           3   
3     100.697424                         UCLA     USA       1995           1   
4     108.862080                    Villanova     USA       1985           1   

   ...  pts   reb  ast  net_rating  oreb_pct 

In [11]:
df['draft_year'] = df['season'].str.split('-').str[0].astype(int)

# Define the target year and the window size
target_year = 1996  # Replace with your desired target year
window_size = 5

# Calculate the start and end years of the window
start_year = target_year - window_size
end_year = target_year

# Filter the dataset to include only records within the 5-year window
filtered_df = df[(df['draft_year'] >= start_year) & (df['draft_year'] <= end_year)]

# Ensure the filtered dataset has at least 100 players
if len(filtered_df) < 100:
    print("There are not enough players within the specified window.")
else:
    # Randomly select 100 players from the filtered dataset
    random.seed(42)  # Set a random seed for reproducibility
    selected_players = random.sample(range(len(filtered_df)), 100)

    # Create a DataFrame containing the selected players
    selected_df = filtered_df.iloc[selected_players]

    # Display the selected players
    print(selected_df)


     Unnamed: 0       player_name team_abbreviation   age  player_height  \
327         327     Tom Gugliotta               MIN  27.0         208.28   
57           57  Hot Rod Williams               PHX  34.0         210.82   
12           12     Emanual Davis               HOU  28.0         195.58   
379         379      Jud Buechler               CHI  29.0         198.12   
140         140    Aaron Williams               VAN  25.0         205.74   
..          ...               ...               ...   ...            ...   
435         435      Martin Lewis               TOR  22.0         198.12   
358         358       Kenny Smith               DEN  32.0         190.50   
236         236    Sedale Threatt               HOU  35.0         187.96   
363         363     Jimmy Carruth               MIL  27.0         208.28   
138         138        A.C. Green               DAL  33.0         205.74   

     player_weight                          college country  draft_year  \
327     108.


## defining Player types    

In [12]:
"""
1 point guard
    ast = 0.6
    ast_pct = 0.4

2 shooting guard
    pts = 0.8
    ast = 0.2

3 small forward
    net_rating = 0.5
    usg_pct = 0.5

4 forward
    pts = 0.4
    oreb_pct = 0.6

5 center
    reb = 0.7
    dreb_pct = 0.3 


"""
weights = {
    "point guard": {"ast": 0.6, "ast_pct": 0.4},
    "shooting guard": {"pts": 0.8, "ast": 0.2},
    "small forward": {"net_rating": 0.5, "usg_pct": 0.5},
    "forward": {"pts": 0.4, "oreb_pct": 0.6},
    "center": {"reb": 0.7, "dreb_pct": 0.3}
}




In [22]:

model = keras.Sequential([
    keras.layers.Input(shape=(8,)),  # 8 input features
    keras.layers.Dense(64, activation='relu'),  # First hidden layer with 64 neurons and ReLU activation
    keras.layers.Dense(64, activation='relu'),  # Second hidden layer with 64 neurons and ReLU activation
    keras.layers.Dense(64, activation='relu'),  # Third hidden layer with 64 neurons and ReLU activation
    keras.layers.Dense(64, activation='relu'),  # Fourth hidden layer with 64 neurons and ReLU activation
    keras.layers.Dense(64, activation='relu'),  # Fifth hidden layer with 64 neurons and ReLU activation
    keras.layers.Dense(num_clusters, activation='softmax')   # Output layer with 5 nodes (one for each player type)
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



C:\Users\josh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Epoch 1/50
385/385 [==============================] - 2s 4ms/step - loss: 0.3268 - accuracy: 0.8781
Epoch 2/50
385/385 [==============================] - 2s 4ms/step - loss: 0.1036 - accuracy: 0.9593
Epoch 3/50
385/385 [==============================] - 2s 4ms/step - loss: 0.1043 - accuracy: 0.9563
Epoch 4/50
385/385 [==============================] - 1s 4ms/step - loss: 0.0950 - accuracy: 0.9605
Epoch 5/50
385/385 [==============================] - 1s 3ms/step - loss: 0.0893 - accuracy: 0.9638
Epoch 6/50
385/385 [==============================] - 2s 4ms/step - loss: 0.0817 - accuracy: 0.9668
Epoch 7/50
385/385 [==============================] - 2s 4ms/step - loss: 0.0865 - accuracy: 0.9628
Epoch 8/50
385/385 [==============================] - 2s 4ms/step - loss: 0.0701 - accuracy: 0.9703
Epoch 9/50
385/385 [==============================] - 2s 4ms/step - loss: 0.0740 - accuracy: 0.9695
Epoch 10/50
385/385 [==============================] - 2s 4ms/step - loss: 0.0707 - accuracy: 0.9711

INFO:tensorflow:Assets written to: player_type_model\assets


In [23]:
import tensorflow as tf

model = tf.keras.models.load_model('player_type_model')
# Replace 'new_input_features' with your new input data
new_input_features = scaler.transform(new_input_features)  # Normalize the new input data
predictions = model.predict(new_input_features)


NameError: name 'new_input_features' is not defined